## RoBERTa introduction and Credits


Twitter-roBERTa-base for Sentiment Analysis

This is a roBERTa-base model trained on ~58M tweets and finetuned for sentiment analysis with the TweetEval benchmark. This model is suitable for English (for a similar multilingual model, see XLM-T).

Reference Paper: TweetEval (Findings of EMNLP 2020).
Git Repo: Tweeteval official repository.
Labels: 0 -> Negative; 1 -> Neutral; 2 -> Positive



reference paper: https://aclanthology.org/2020.findings-emnlp.148/


the code used in this notebook as been extracted from the following repository: https://huggingface.co/blog/bert-101#7-how-to-get-started-using-bert


```
# from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)


from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)
res = []
for text in texts:
  try:
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

  # # TF
  # model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
  # model.save_pretrained(MODEL)

  # text = "Good night 😊"
  # encoded_input = tokenizer(text, return_tensors='tf')
  # output = model(encoded_input)
  # scores = output[0][0].numpy()
  # scores = softmax(scores)
    d = {"text": text}
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    for i in range(scores.shape[0]):
        l = labels[ranking[i]]
        s = scores[ranking[i]]
        d[f"{l}"] = f'{np.round(float(s),4)}'
    res.append(d)
  except:
    d = {"text": text, "labels":"NA"}
    res.append(d)

from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Emotions:
# anger, joy, sadness, and optimism​

task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

text = "Celebrating my promotion 😎"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)

# text = "Celebrating my promotion 😎"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")
```



## Functions to run




In [ ]:
# Preprocessing functions

#original function of the repository of Roberta-pretrained
def preprocess(text):
  new_text = []
  for t in text.split(" "):
    t = 'http' if t.startswith('http') else t
    new_text.append(t)
  return " ".join(new_text)

#modified function to fit our necessities
import re
def preprocess1(text):
   cleaned_text = re.sub(r'http[s]?://\S+', '', text)
   cleaned_text_no_specials = re.sub(r'[^a-zA-Z0-9\s]', '', cleaned_text)
   return cleaned_text_no_specials


In [ ]:
# function to truncate the posts/comments whose text is too long
def truncate_doc(doc, max_len=350):
  tokens = doc.split()
  truncated_tokens = tokens[:max_len]
  truncated_doc = " ".join(truncated_tokens)
  return truncated_doc

In [ ]:
# Run this only once in the entire notebook, if run twice it will provide an error. Interrup run time and rerun everything if needed
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
# function of emotion analysis

def emotion_analysis(texts):
  mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
  with urllib.request.urlopen(mapping_link) as f:
      html = f.read().decode('utf-8').split("\n")
      csvreader = csv.reader(html, delimiter='\t')
  labels = [row[1] for row in csvreader if len(row) > 1]

  model = AutoModelForSequenceClassification.from_pretrained(MODEL)
  model.save_pretrained(MODEL)
  res = []
  for text in texts:
    try:
      encoded_input = tokenizer(text, return_tensors='pt')
      output = model(**encoded_input)
      scores = output[0][0].detach().numpy()
      scores = softmax(scores)
      d = {"text": text}
      ranking = np.argsort(scores)
      ranking = ranking[::-1]
      for i in range(scores.shape[0]):
        l = labels[ranking[i]]
        s = scores[ranking[i]]
        d[f"{l}"] = f'{np.round(float(s),4)}'
      res.append(d)

    except:
      try:
        text = truncate_doc(text)
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        d = {"text": text}
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        for i in range(scores.shape[0]):
          l = labels[ranking[i]]
          s = scores[ranking[i]]
          d[f"{l}"] = f'{np.round(float(s),4)}'
        res.append(d)
      except:
        d = {"text": text, "labels":"Error length"}
        res.append(d)

  return res

## **Objective** **of** **the** **notebook**: Analyze the emotion in the post vs the reply after the comment
(we will use only the posts which have comment and reply)

##Depression Comments and replies

In [ ]:
#load data
import pandas as pd
df1 = pd.read_csv("depression_comments_replies.csv")

In [ ]:
#cleaning
df1['Post'] = df1['title'] + ' ' + df1['text']
df1['Post_cleaned'] = df1.Post.apply(preprocess1)
df1['Comment_cleaned'] = df1.Comment.apply(preprocess1)
df1['Answer_cleaned'] = df1.Answer.apply(preprocess1)

In [ ]:
depression_pca = df1[['Post','Post_cleaned','Comment','Comment_cleaned','Answer','Answer_cleaned']]
depression_pca.head()

In [ ]:
texts_p = [t for t in depression_pca['Post_cleaned']]
texts_c = [t for t in depression_pca['Comment_cleaned']]
texts_a = [t for t in depression_pca['Answer_cleaned']]

In [ ]:
#emotion
import pandas as pd

res_e_p = emotion_analysis(texts_p)
res_e_c = emotion_analysis(texts_c)
res_e_a = emotion_analysis(texts_a)
emotion_depression_pp = pd.DataFrame(res_e_p)
emotion_depression_c= pd.DataFrame(res_e_c)
emotion_depression_a = pd.DataFrame(res_e_a)

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
emotion_depression_pp.to_csv("emotion_depression_pp")
emotion_depression_c.to_csv("emotion_depression_c")
emotion_depression_a.to_csv("emotion_depression_a")

total dataset creation

In [ ]:
emotion_depression_pp.rename(columns={'text': 'post','joy':'joy_p','optimism':'optimism_p', 'anger':'anger_p', 'sadness':'sadness_p' }, inplace=True)
emotion_depression_c.rename(columns={'text': 'comment','joy':'joy_c', 'optimism':'optimism_c', 'anger':'anger_c', 'sadness':'sadness_c'}, inplace=True)
emotion_depression_a.rename(columns={'text': 'answer','joy':'joy_a', 'optimism':'optimism_a', 'anger':'anger_a', 'sadness':'sadness_a'}, inplace=True)
concatenated_df = pd.concat([emotion_depression_pp, emotion_depression_c, emotion_depression_a], axis=1)
concatenated_df.to_csv("emotion_depression_pca_scores")

emotion difference

In [ ]:
emotion_df = pd.read_csv("emotion_depression_pca_scores")
emotion_df.head()
emotion_df["joy_diff"] = emotion_df['joy_p'] - emotion_df['joy_a']
emotion_df["anger_diff"] = emotion_df['anger_p'] - emotion_df['anger_a']
emotion_df["sadness_diff"] = emotion_df['sadness_p'] - emotion_df['sadness_a']
emotion_df["optimism_diff"] = emotion_df['optimism_p'] - emotion_df['optimism_a']

In [ ]:
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
print(avg_diff_joy)
#the replies are more likely to be labelled "joy" than the posts by 0.13

avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
print(avg_diff_anger)
#the replies are more likely to be labelled "anger" than the posts by 0.023

avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
print(avg_diff_sadness)
#the replies are less likely to be labelled "sadness" than the posts by 0.338

avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_optimism)
#the replies are more likely to be labelled "optimsm" than the posts by 0.17

#-0.13585808338332336
#-0.023732123575284942
#0.33826898620275947
#-0.17868080383923216

-0.13585808338332336
-0.023732123575284942
0.33826898620275947
-0.17868080383923216


## Anxiety Comments and Replies

In [ ]:
#load data
import pandas as pd
df1 = pd.read_csv("anxiety_comments_replies.csv")

In [ ]:
#cleaning
df1['Post'] = df1['title'] + ' ' + df1['text']
df1['Post'] = df1['Post'].astype(str)
df1['Post_cleaned'] = df1.Post.apply(preprocess1)
df1['Comment'] = df1['Comment'].astype(str)
df1['Comment_cleaned'] = df1.Comment.apply(preprocess1)
df1['Answer'] = df1['Answer'].astype(str)
df1['Answer_cleaned'] = df1.Answer.apply(preprocess1)

In [ ]:
anxiety_pca = df1[['Post','Post_cleaned','Comment','Comment_cleaned','Answer','Answer_cleaned']]
anxiety_pca.head()

,Post,Post_cleaned,Comment,Comment_cleaned,Answer,Answer_cleaned
0,"Professor here, if you have missed class, plea...",Professor here if you have missed class please...,This is honestly so helpful. Thank you,This is honestly so helpful Thank you,No problem. I suffer from anxiety as well and ...,No problem I suffer from anxiety as well and \...
1,"Professor here, if you have missed class, plea...",Professor here if you have missed class please...,So funny because I am also a professor and I a...,So funny because I am also a professor and I a...,I kept reading the posts and thinking a differ...,I kept reading the posts and thinking a differ...
2,"Professor here, if you have missed class, plea...",Professor here if you have missed class please...,I've had professors straight up make fun of me...,Ive had professors straight up make fun of me ...,That’s some bullshit.,Thats some bullshit
3,"Professor here, if you have missed class, plea...",Professor here if you have missed class please...,I wish you were my professor. I get so anxious...,I wish you were my professor I get so anxious ...,Even if we are super caring people we rarely n...,Even if we are super caring people we rarely n...
4,"Professor here, if you have missed class, plea...",Professor here if you have missed class please...,It's great that there are teachers/professors ...,Its great that there are teachersprofessors li...,That’s absurd. I’m sorry that happened.,Thats absurd Im sorry that happened


In [ ]:
texts_p = [t for t in anxiety_pca['Post_cleaned']]
texts_c = [t for t in anxiety_pca['Comment_cleaned']]
texts_a = [t for t in anxiety_pca['Answer_cleaned']]

In [ ]:
import pandas as pd

res_e_p = emotion_analysis(texts_p)
res_e_c = emotion_analysis(texts_c)
res_e_a = emotion_analysis(texts_a)
emotion_anxiety_pp = pd.DataFrame(res_e_p)
emotion_anxiety_c= pd.DataFrame(res_e_c)
emotion_anxiety_a = pd.DataFrame(res_e_a)

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
emotion_anxiety_pp.to_csv("emotion_anxiety_pp")
emotion_anxiety_c.to_csv("emotion_anxiety_c")
emotion_anxiety_a.to_csv("emotion_anxiety_a")

Total dataset creation

In [ ]:
emotion_anxiety_pp.rename(columns={'text': 'post','joy':'joy_p','optimism':'optimism_p', 'anger':'anger_p', 'sadness':'sadness_p' }, inplace=True)
emotion_anxiety_c.rename(columns={'text': 'comment','joy':'joy_c', 'optimism':'optimism_c', 'anger':'anger_c', 'sadness':'sadness_c'}, inplace=True)
emotion_anxiety_a.rename(columns={'text': 'answer','joy':'joy_a', 'optimism':'optimism_a', 'anger':'anger_a', 'sadness':'sadness_a'}, inplace=True)
concatenated_df = pd.concat([emotion_anxiety_pp, emotion_anxiety_c, emotion_anxiety_a], axis=1)
concatenated_df.to_csv("emotion_anxiety_pca_scores")

Emotion difference

In [ ]:
emotion_df = pd.read_csv("emotion_anxiety_pca_scores")
emotion_df.head()
emotion_df["joy_diff"] = emotion_df['joy_p'] - emotion_df['joy_a']
emotion_df["anger_diff"] = emotion_df['anger_p'] - emotion_df['anger_a']
emotion_df["sadness_diff"] = emotion_df['sadness_p'] - emotion_df['sadness_a']
emotion_df["optimism_diff"] = emotion_df['optimism_p'] - emotion_df['optimism_a']

In [ ]:
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
print(avg_diff_joy)
#the replies are more likely to be labelled "joy" than the posts by 0.07

avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
print(avg_diff_anger)
#the replies are more likely to be labelled "anger" than the posts by 0.006

avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
print(avg_diff_sadness)
#the replies are less likely to be labelled "sadness" than the posts by 0.187

avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_optimism)
#the replies are more likely to be labelled "optimsm" than the posts by 0.109

#-0.0717700816629547
#-0.006097141796585003
#0.1868859688195991
#-0.10902234595397178

-0.0717700816629547
-0.006097141796585003
0.1868859688195991
-0.10902234595397178


In [ ]:
emotion_df.to_csv("emotion_anxiety.csv")

## Stress Comments and replies

In [ ]:
#load data
import pandas as pd
df1 = pd.read_csv("stress_comments_replies.csv")

In [ ]:
#cleaning
df1['Post'] = df1['title'] + ' ' + df1['text']
df1['Post_cleaned'] = df1.Post.apply(preprocess1)
df1['Comment_cleaned'] = df1.Comment.apply(preprocess1)
df1['Answer_cleaned'] = df1.Answer.apply(preprocess1)

In [ ]:
stress_pca = df1[['Post','Post_cleaned','Comment','Comment_cleaned','Answer','Answer_cleaned']]
stress_pca.head()

,Post,Post_cleaned,Comment,Comment_cleaned,Answer,Answer_cleaned
0,"To everyone with stress, here you go a online ...",To everyone with stress here you go a online b...,"This made me smile, thanks :)",This made me smile thanks,Np,Np
1,"To everyone with stress, here you go a online ...",To everyone with stress here you go a online b...,"To everyone with stress, here you go a online ...",To everyone with stress here you go a online b...,&gt;!5 e !&lt;,gt5 e lt
2,Does anyone else feel CONSTANTLY stressed? My ...,Does anyone else feel CONSTANTLY stressed My w...,It sounds like you want to achieve a state whe...,It sounds like you want to achieve a state whe...,That actually sounds pretty fitting! Thank you...,That actually sounds pretty fitting Thank you ...
3,Does anyone else feel CONSTANTLY stressed? My ...,Does anyone else feel CONSTANTLY stressed My w...,yes ALL the time. its super draining imo :( \n...,yes ALL the time its super draining imo \ndis...,It’s SO refreshing to hear someone else feels ...,Its SO refreshing to hear someone else feels t...
4,Does anyone else feel CONSTANTLY stressed? My ...,Does anyone else feel CONSTANTLY stressed My w...,What are your thoughts about relaxing? Do you ...,What are your thoughts about relaxing Do you f...,Yes!! I definitely was told growing up that re...,Yes I definitely was told growing up that rela...


In [ ]:
texts_p = [t for t in stress_pca['Post_cleaned']]
texts_c = [t for t in stress_pca['Comment_cleaned']]
texts_a = [t for t in stress_pca['Answer_cleaned']]

In [ ]:
import pandas as pd

res_e_p = emotion_analysis(texts_p)
res_e_c = emotion_analysis(texts_c)
res_e_a = emotion_analysis(texts_a)
emotion_stress_pp = pd.DataFrame(res_e_p)
emotion_stress_c= pd.DataFrame(res_e_c)
emotion_stress_a = pd.DataFrame(res_e_a)

In [ ]:
emotion_stress_pp.to_csv("emotion_stress_pp")
emotion_stress_c.to_csv("emotion_stress_c")
emotion_stress_a.to_csv("emotion_stress_a")

Total dataset creation

In [ ]:
emotion_stress_pp.rename(columns={'text': 'post','joy':'joy_p','optimism':'optimism_p', 'anger':'anger_p', 'sadness':'sadness_p' }, inplace=True)
emotion_stress_c.rename(columns={'text': 'comment','joy':'joy_c', 'optimism':'optimism_c', 'anger':'anger_c', 'sadness':'sadness_c'}, inplace=True)
emotion_stress_a.rename(columns={'text': 'answer','joy':'joy_a', 'optimism':'optimism_a', 'anger':'anger_a', 'sadness':'sadness_a'}, inplace=True)
concatenated_df = pd.concat([emotion_stress_pp, emotion_stress_c, emotion_stress_a], axis=1)
concatenated_df.to_csv("emotion_stress_pca_scores")

Emotion difference

In [ ]:
emotion_df = pd.read_csv("emotion_stress_pca_scores")
emotion_df.head()
emotion_df["joy_diff"] = emotion_df['joy_p'] - emotion_df['joy_a']
emotion_df["anger_diff"] = emotion_df['anger_p'] - emotion_df['anger_a']
emotion_df["sadness_diff"] = emotion_df['sadness_p'] - emotion_df['sadness_a']
emotion_df["optimism_diff"] = emotion_df['optimism_p'] - emotion_df['optimism_a']

In [ ]:
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
print(avg_diff_joy)
#the replies are more likely to be labelled "joy" than the posts by 0.166

avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
print(avg_diff_anger)
#the replies are less likely to be labelled "anger" than the posts by 0.106

avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
print(avg_diff_sadness)
#the replies are less likely to be labelled "sadness" than the posts by 0.251

avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_optimism)
#the replies are more likely to be labelled "optimsm" than the posts by 0.190

#-0.16598676646706587
#0.10563640718562875
#0.2506323353293413
#-0.1902803592814371

-0.16598676646706587
0.10563640718562875
0.2506323353293413
-0.1902803592814371


In [ ]:
emotion_df.to_csv('emotion_stress.csv')

## Anger Comments and Replies

In [ ]:
#load data
import pandas as pd
df1 = pd.read_csv("anger_comments_replies.csv")

In [ ]:
#cleaning
df1['Post'] = df1['title'] + ' ' + df1['text']
df1['Post_cleaned'] = df1.Post.apply(preprocess1)
df1['Comment_cleaned'] = df1.Comment.apply(preprocess1)
df1['Answer_cleaned'] = df1.Answer.apply(preprocess1)

In [ ]:
anger_pca = df1[['Post','Post_cleaned','Comment','Comment_cleaned','Answer','Answer_cleaned']]
anger_pca.head()

,Post,Post_cleaned,Comment,Comment_cleaned,Answer,Answer_cleaned
0,Not a single person has upvoted anything I've ...,Not a single person has upvoted anything Ive p...,You are on Reddit for the wrong reason.,You are on Reddit for the wrong reason,Thats fair,Thats fair
1,Not a single person has upvoted anything I've ...,Not a single person has upvoted anything Ive p...,well hey. Happy cake day.,well hey Happy cake day,Its a bit late but thanks :&gt;,Its a bit late but thanks gt
2,Not a single person has upvoted anything I've ...,Not a single person has upvoted anything Ive p...,Happy late cake day! Hope your day is going well,Happy late cake day Hope your day is going well,Thanks mate,Thanks mate
3,Not a single person has upvoted anything I've ...,Not a single person has upvoted anything Ive p...,Happy cake day!! I'm just one person but I hop...,Happy cake day Im just one person but I hope i...,Thanks. Really.\n\nEdit: Why the fuck are peop...,Thanks Really\n\nEdit Why the fuck are people ...
4,Not a single person has upvoted anything I've ...,Not a single person has upvoted anything Ive p...,Happy cake day! Sorry about the missing upvotes.,Happy cake day Sorry about the missing upvotes,Who in the name of fuck downvoted this comment,Who in the name of fuck downvoted this comment


In [ ]:
texts_p = [t for t in anger_pca['Post_cleaned']]
texts_c = [t for t in anger_pca['Comment_cleaned']]
texts_a = [t for t in anger_pca['Answer_cleaned']]

In [ ]:
import pandas as pd

res_p = emotion_analysis(texts_p)
res_c = emotion_analysis(texts_c)
res_a = emotion_analysis(texts_a)
emotion_anger_pp = pd.DataFrame(res_p)
emotion_anger_c= pd.DataFrame(res_c)
emotion_anger_a = pd.DataFrame(res_a)

In [ ]:
emotion_anger_pp.to_csv("emotion_anger_pp")
emotion_anger_c.to_csv("emotion_anger_c")
emotion_anger_a.to_csv("emotion_anger_a")

total dataset creation

In [ ]:
emotion_anger_pp.rename(columns={'text': 'post','joy':'joy_p','optimism':'optimism_p', 'anger':'anger_p', 'sadness':'sadness_p' }, inplace=True)
emotion_anger_c.rename(columns={'text': 'comment','joy':'joy_c', 'optimism':'optimism_c', 'anger':'anger_c', 'sadness':'sadness_c'}, inplace=True)
emotion_anger_a.rename(columns={'text': 'answer','joy':'joy_a', 'optimism':'optimism_a', 'anger':'anger_a', 'sadness':'sadness_a'}, inplace=True)
concatenated_df = pd.concat([emotion_anger_pp, emotion_anger_c, emotion_anger_a], axis=1)
concatenated_df.to_csv("emotion_anger_pca_scores")

emotion difference

In [ ]:
emotion_df = pd.read_csv("emotion_anger_pca_scores")
emotion_df.head()
emotion_df["joy_diff"] = emotion_df['joy_p'] - emotion_df['joy_a']
emotion_df["anger_diff"] = emotion_df['anger_p'] - emotion_df['anger_a']
emotion_df["sadness_diff"] = emotion_df['sadness_p'] - emotion_df['sadness_a']
emotion_df["optimism_diff"] = emotion_df['optimism_p'] - emotion_df['optimism_a']

In [ ]:
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
print(avg_diff_joy)
#the replies are more likely to be labelled "joy" than the posts by 0.149

avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
print(avg_diff_anger)
#the replies are less to be labelled "anger" than the posts by 0.452

avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
print(avg_diff_sadness)
#the replies are more likely to be labelled "sadness" than the posts by 0.152

avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_optimism)
#the replies are more likely to be labelled "optimsm" than the posts by 0.150

#-0.1491208695652174
#0.4523078260869565
#-0.15295282608695654
#-0.15023876811594203

-0.1491208695652174
0.4523078260869565
-0.15295282608695654
-0.15023876811594203


In [ ]:
emotion_df.to_csv("emotion_anger.csv")

## Relationship Comments and Replies

In [ ]:
#load data
import pandas as pd
df1 = pd.read_csv("relationship_comments_replies.csv")

In [ ]:
#cleaning
df1['Post'] = df1['title'] + ' ' + df1['text']
df1['Post_cleaned'] = df1.Post.apply(preprocess1)
df1['Comment_cleaned'] = df1.Comment.apply(preprocess1)
df1['Answer_cleaned'] = df1.Answer.apply(preprocess1)

In [ ]:
relationship_pca = df1[['Post','Post_cleaned','Comment','Comment_cleaned','Answer','Answer_cleaned']]
relationship_pca.head()

,Post,Post_cleaned,Comment,Comment_cleaned,Answer,Answer_cleaned
0,To whoever needs to hear this. He made you fee...,To whoever needs to hear this He made you feel...,thank you for this,thank you for this,"Just remember, you are more than what one pers...",Just remember you are more than what one perso...
1,To whoever needs to hear this. He made you fee...,To whoever needs to hear this He made you feel...,This is so true. Why do his actions of treatin...,This is so true Why do his actions of treating...,i’m sorry it took me so long to see this. he m...,im sorry it took me so long to see this he mak...
2,To whoever needs to hear this. He made you fee...,To whoever needs to hear this He made you feel...,needed this. thanks OP,needed this thanks OP,You are worth more than even you know. Never l...,You are worth more than even you know Never le...
3,To whoever needs to hear this. He made you fee...,To whoever needs to hear this He made you feel...,"I needed to see this, even if i had to change ...",I needed to see this even if i had to change i...,i only said he because i felt if i made it gen...,i only said he because i felt if i made it gen...
4,To whoever needs to hear this. He made you fee...,To whoever needs to hear this He made you feel...,:( I can’t escape the thoughts that came after...,I cant escape the thoughts that came afterwar...,just remember you already know you can do it w...,just remember you already know you can do it w...


In [ ]:
texts_p = [t for t in relationship_pca['Post_cleaned']]
texts_c = [t for t in relationship_pca['Comment_cleaned']]
texts_a = [t for t in relationship_pca['Answer_cleaned']]

In [ ]:
import pandas as pd

res_p = emotion_analysis(texts_p)
res_c = emotion_analysis(texts_c)
res_a = emotion_analysis(texts_a)
emotion_relationship_pp = pd.DataFrame(res_p)
emotion_relationship_c= pd.DataFrame(res_c)
emotion_relationship_a = pd.DataFrame(res_a)

In [ ]:
emotion_relationship_pp.to_csv("emotion_relationship_pp")
emotion_relationship_c.to_csv("emotion_relationship_c")
emotion_relationship_a.to_csv("emotion_relationship_a")

Total dataset creation

In [ ]:
emotion_relationship_pp.rename(columns={'text': 'post','joy':'joy_p','optimism':'optimism_p', 'anger':'anger_p', 'sadness':'sadness_p' }, inplace=True)
emotion_relationship_c.rename(columns={'text': 'comment','joy':'joy_c', 'optimism':'optimism_c', 'anger':'anger_c', 'sadness':'sadness_c'}, inplace=True)
emotion_relationship_a.rename(columns={'text': 'answer','joy':'joy_a', 'optimism':'optimism_a', 'anger':'anger_a', 'sadness':'sadness_a'}, inplace=True)
concatenated_df = pd.concat([emotion_relationship_pp, emotion_relationship_c, emotion_relationship_a], axis=1)
concatenated_df.to_csv("emotion_relationship_pca_scores")

emotion difference

In [ ]:
emotion_df = pd.read_csv("emotion_relationship_pca_scores")
emotion_df.head()
emotion_df["joy_diff"] = emotion_df['joy_p'] - emotion_df['joy_a']
emotion_df["anger_diff"] = emotion_df['anger_p'] - emotion_df['anger_a']
emotion_df["sadness_diff"] = emotion_df['sadness_p'] - emotion_df['sadness_a']
emotion_df["optimism_diff"] = emotion_df['optimism_p'] - emotion_df['optimism_a']

In [ ]:
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
print(avg_diff_joy)
#the replies are more likely to be labelled "joy" than the posts by 0.151

avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
print(avg_diff_anger)
#the replies are less likely to be labelled "anger" than the posts by 0.187

avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
print(avg_diff_sadness)
#the replies are less likely to be labelled "sadness" than the posts by 0.072

avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_optimism)
#the replies are more likely to be labelled "optimsm" than the posts by 0.109

#-0.15065051975051974
#0.18692827442827445
#0.07247567567567569
#-0.10874511434511434

-0.15065051975051974
0.18692827442827445
0.07247567567567569
-0.10874511434511434


In [ ]:
emotion_df.to_csv("emotion_relationship.csv")

## Emotion analysis all files comments and replies

In [ ]:
#importing all the datasets and merging them together
import pandas as pd
f0 = pd.read_csv("anger_comments_replies.csv")
f0['group'] = 'anger'
f1 = pd.read_csv("anxiety_comments_replies.csv")
f1['group'] = 'anxiety'
f2 = pd.read_csv("depression_comments_replies.csv")
f2['group'] = 'depression'
f3 = pd.read_csv("relationship_comments_replies.csv")
f3['group'] = 'relationship'
f4 = pd.read_csv("stress_comments_replies.csv")
f4['group'] = 'stress'
df1 = pd.concat([f0, f1, f2, f3, f4], axis=0)
df1.head()
#print(len(f0), len(f1), len(f2), len(f3), len(f4), len(f0)+len(f1)+len(f2)+len(f3)+len(f4),len(file_tot))

,title,text,author,Comment,Answer,group
0,Not a single person has upvoted anything I've ...,I know this is such a stupid pet peeve but still.,[deleted],You are on Reddit for the wrong reason.,Thats fair,anger
1,Not a single person has upvoted anything I've ...,I know this is such a stupid pet peeve but still.,[deleted],well hey. Happy cake day.,Its a bit late but thanks :&gt;,anger
2,Not a single person has upvoted anything I've ...,I know this is such a stupid pet peeve but still.,[deleted],Happy late cake day! Hope your day is going well,Thanks mate,anger
3,Not a single person has upvoted anything I've ...,I know this is such a stupid pet peeve but still.,[deleted],Happy cake day!! I'm just one person but I hop...,Thanks. Really.\n\nEdit: Why the fuck are peop...,anger
4,Not a single person has upvoted anything I've ...,I know this is such a stupid pet peeve but still.,[deleted],Happy cake day! Sorry about the missing upvotes.,Who in the name of fuck downvoted this comment,anger


In [ ]:
df1['Post'] = df1['title'] + ' ' + df1['text']
df1['Post_cleaned'] = df1.Post.apply(preprocess1)
df1['Answer_cleaned'] = df1.Answer.apply(preprocess1)
df1['Comment'] = df1['Comment'].astype(str)
df1['Comment_cleaned'] = df1.Comment.apply(preprocess1)

In [ ]:
all_pca = df1[['Post','Post_cleaned','Comment','Comment_cleaned','Answer','Answer_cleaned']]
all_pca.head()

,Post,Post_cleaned,Comment,Comment_cleaned,Answer,Answer_cleaned
0,Not a single person has upvoted anything I've ...,Not a single person has upvoted anything Ive p...,You are on Reddit for the wrong reason.,You are on Reddit for the wrong reason,Thats fair,Thats fair
1,Not a single person has upvoted anything I've ...,Not a single person has upvoted anything Ive p...,well hey. Happy cake day.,well hey Happy cake day,Its a bit late but thanks :&gt;,Its a bit late but thanks gt
2,Not a single person has upvoted anything I've ...,Not a single person has upvoted anything Ive p...,Happy late cake day! Hope your day is going well,Happy late cake day Hope your day is going well,Thanks mate,Thanks mate
3,Not a single person has upvoted anything I've ...,Not a single person has upvoted anything Ive p...,Happy cake day!! I'm just one person but I hop...,Happy cake day Im just one person but I hope i...,Thanks. Really.\n\nEdit: Why the fuck are peop...,Thanks Really\n\nEdit Why the fuck are people ...
4,Not a single person has upvoted anything I've ...,Not a single person has upvoted anything Ive p...,Happy cake day! Sorry about the missing upvotes.,Happy cake day Sorry about the missing upvotes,Who in the name of fuck downvoted this comment,Who in the name of fuck downvoted this comment


In [ ]:
texts_p = [t for t in all_pca['Post_cleaned']]
texts_c = [t for t in all_pca['Comment_cleaned']]
texts_a = [t for t in all_pca['Answer_cleaned']]

In [ ]:
# assessing emotion on the whole dataset
import pandas as pd

res_e_p = emotion_analysis(texts_p)
res_e_c = emotion_analysis(texts_c)
res_e_a = emotion_analysis(texts_a)
emotion_all_pp = pd.DataFrame(res_e_p)
emotion_all_c= pd.DataFrame(res_e_c)
emotion_all_a = pd.DataFrame(res_e_a)

In [ ]:
emotion_all_pp.to_csv("emotion_all_pp")
emotion_all_c.to_csv("emotion_all_c")
emotion_all_a.to_csv("emotion_all_a")

total dataset creation

In [ ]:
emotion_all_pp.rename(columns={'text': 'post','joy':'joy_p','optimism':'optimism_p', 'anger':'anger_p', 'sadness':'sadness_p' }, inplace=True)
emotion_all_c.rename(columns={'text': 'comment','joy':'joy_c', 'optimism':'optimism_c', 'anger':'anger_c', 'sadness':'sadness_c'}, inplace=True)
emotion_all_a.rename(columns={'text': 'answer','joy':'joy_a', 'optimism':'optimism_a', 'anger':'anger_a', 'sadness':'sadness_a'}, inplace=True)
concatenated_df = pd.concat([emotion_all_pp, emotion_all_c, emotion_all_a], axis=1)
concatenated_df.to_csv("emotion_all_pca_scores")

## Final emotion difference analysis

In [ ]:
emotion_df = pd.read_csv("emotion_all_pca_scores")
emotion_df.head()
emotion_df["joy_diff"] = emotion_df['joy_p'] - emotion_df['joy_a']
emotion_df["anger_diff"] = emotion_df['anger_p'] - emotion_df['anger_a']
emotion_df["sadness_diff"] = emotion_df['sadness_p'] - emotion_df['sadness_a']
emotion_df["optimism_diff"] = emotion_df['optimism_p'] - emotion_df['optimism_a']

In [ ]:
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
print(avg_diff_joy)
#the replies are more likely to be labelled "joy" than the posts by 0.117

avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
print(avg_diff_anger)
#the replies are less likely to be labelled "anger" than the posts by 0.041

avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
print(avg_diff_sadness)
#the replies are less likely to be labelled "sadness" than the posts by 0.229

avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_optimism)
#the replies are more likely to be labelled "optimsm" than the posts by 0.148

#-0.11739236942980354
#0.04099598107331097
#0.22488026473406802
#-0.148484217776713

In [ ]:
emotion_df['max_p'] = emotion_df[['anger_p','sadness_p','optimism_p','joy_p']].idxmax(axis=1)
emotion_df['max_c'] = emotion_df[['anger_c','sadness_c','optimism_c','joy_c']].idxmax(axis=1)
emotion_df['max_a'] = emotion_df[['anger_a','sadness_a','optimism_a','joy_a']].idxmax(axis=1)
emotion_df.to_csv("dataset_arianna.csv")

In [ ]:
import pandas as pd
emotion_df = pd.read_csv("dataset_r_emotion_all.csv")

In [ ]:
emotion_df_joy = emotion_df[emotion_df.max_c == "joy_c"]
emotion_df_optimism = emotion_df[emotion_df.max_c == "optimism_c"]
emotion_df_anger = emotion_df[emotion_df.max_c == "anger_c"]
emotion_df_sadness = emotion_df[emotion_df.max_c == "sadness_c"]
emotion_df_joy.to_csv("emotion_df_joy.csv")
emotion_df_optimism.to_csv("emotion_df_optimism.csv")
emotion_df_anger.to_csv("emotion_df_anger.csv")
emotion_df_sadness.to_csv("emotion_df_sadness.csv")

In [ ]:
#FOR JOY COMMENTS
emotion_df = emotion_df_joy
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
print(avg_diff_joy)

avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
print(avg_diff_anger)

avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
print(avg_diff_sadness)

avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_optimism)

#-0.14632171581769435
#0.00928374664879357
#0.21510740616621984
#-0.07806906836461126

-0.14632171581769435
0.00928374664879357
0.21510740616621984
-0.07806906836461126


In [ ]:
#FOR SADNESS COMMENTS
emotion_df = emotion_df_sadness
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
print(avg_diff_joy)

avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
print(avg_diff_anger)

avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
print(avg_diff_sadness)

avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_optimism)

#-0.10193340127388535
#0.01598081528662421
#0.2463907770700637
#-0.1604372611464968

-0.10193340127388535
0.01598081528662421
0.2463907770700637
-0.1604372611464968


In [ ]:
#FOR OPTIMISM COMMENTS
emotion_df = emotion_df_optimism
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
print(avg_diff_joy)

avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
print(avg_diff_anger)

avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
print(avg_diff_sadness)

avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_optimism)

#-0.15426796417499136
#0.04701143644505685
#0.326986117809163
#-0.2197287633482604

-0.15426796417499136
0.04701143644505685
0.326986117809163
-0.2197287633482604


In [ ]:
#FOR ANGER COMMENTS
emotion_df = emotion_df_anger
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
print(avg_diff_joy)

avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
print(avg_diff_anger)

avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
print(avg_diff_sadness)

avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_optimism)

#-0.09308380662609872
#0.13348296146044628
#0.07752268424611224
#-0.11792741717376605

-0.09308380662609872
0.13348296146044628
0.07752268424611224
-0.11792741717376605


In [ ]:
#FOR COMMENTS joy AND DIFFERENT TYPES OF POSTS
emotion_df_joy_joy = emotion_df_joy[emotion_df_joy.max_p == "joy_p"]
emotion_df_joy_opti = emotion_df_joy[emotion_df_joy.max_p == "optimism_p"]
emotion_df_joy_anger = emotion_df_joy[emotion_df_joy.max_p == "anger_p"]
emotion_df_joy_sad = emotion_df_joy[emotion_df_joy.max_p == "sadness_p"]

emotion_df = emotion_df_joy_joy
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_joy, avg_diff_anger, avg_diff_sadness, avg_diff_optimism)

emotion_df = emotion_df_joy_opti
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_joy, avg_diff_anger, avg_diff_sadness, avg_diff_optimism)

emotion_df = emotion_df_joy_anger
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_joy, avg_diff_anger, avg_diff_sadness, avg_diff_optimism)

emotion_df = emotion_df_joy_sad
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_joy, avg_diff_anger, avg_diff_sadness, avg_diff_optimism)

"""
0.19655506035283193 -0.051295728876508816 -0.024953203342618382 -0.12030436397400184
-0.3478850340136055 -0.04059024943310658 0.059132199546485256 0.32932630385487527
-0.30871375838926174 0.5513771812080537 -0.07720805369127517 -0.16545637583892617
-0.34524404455869756 -0.054380377035132814 0.5704248500428448 -0.17079451585261352"""

In [ ]:
#FOR COMMENTS anger AND DIFFERENT TYPES OF POSTS
emotion_df_anger_joy = emotion_df_anger[emotion_df_anger.max_p == "joy_p"]
emotion_df_anger_opti = emotion_df_anger[emotion_df_anger.max_p == "optimism_p"]
emotion_df_anger_anger = emotion_df_anger[emotion_df_anger.max_p == "anger_p"]
emotion_df_anger_sad = emotion_df_anger[emotion_df_anger.max_p == "sadness_p"]

emotion_df = emotion_df_anger_joy
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_joy, avg_diff_anger, avg_diff_sadness, avg_diff_optimism)

emotion_df = emotion_df_anger_opti
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_joy, avg_diff_anger, avg_diff_sadness, avg_diff_optimism)

emotion_df = emotion_df_anger_anger
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_joy, avg_diff_anger, avg_diff_sadness, avg_diff_optimism)

emotion_df = emotion_df_anger_sad
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_joy, avg_diff_anger, avg_diff_sadness, avg_diff_optimism)

"""0.3390670658682634 -0.15217904191616768 -0.14875329341317364 -0.03813652694610778
-0.06483456790123457 -0.14013086419753087 -0.1624283950617284 0.367379012345679
-0.12603805496828754 0.4402063424947146 -0.1800474982381959 -0.1341243128964059
-0.11471411947245926 -0.14976392552366175 0.40558417377812256 -0.14111388673390227"""

In [ ]:
#FOR COMMENTS sadness AND DIFFERENT TYPES OF POSTS
emotion_df_sad_joy = emotion_df_sadness[emotion_df_sadness.max_p == "joy_p"]
emotion_df_sad_opti = emotion_df_sadness[emotion_df_sadness.max_p == "optimism_p"]
emotion_df_sad_anger = emotion_df_sadness[emotion_df_sadness.max_p == "anger_p"]
emotion_df_sad_sad = emotion_df_sadness[emotion_df_sadness.max_p == "sadness_p"]

emotion_df = emotion_df_sad_joy
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_joy, avg_diff_anger, avg_diff_sadness, avg_diff_optimism)

emotion_df = emotion_df_sad_opti
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_joy, avg_diff_anger, avg_diff_sadness, avg_diff_optimism)

emotion_df = emotion_df_sad_anger
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_joy, avg_diff_anger, avg_diff_sadness, avg_diff_optimism)

emotion_df = emotion_df_sad_sad
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_joy, avg_diff_anger, avg_diff_sadness, avg_diff_optimism)

"""0.4291587557603686 -0.10008824884792626 -0.2092052995391705 -0.11986336405529951
-0.05275722713864307 -0.054498525073746314 -0.12161828908554574 0.22886961651917404
-0.13337434497816592 0.584313209606987 -0.2889262008733624 -0.16201790393013096
-0.137376481090732 -0.05646294432721961 0.37832295081967215 -0.18448144781691286"""

In [ ]:
#FOR COMMENTS optimism AND DIFFERENT TYPES OF POSTS
emotion_df_opti_joy = emotion_df_optimism[emotion_df_optimism.max_p == "joy_p"]
emotion_df_opti_opti = emotion_df_optimism[emotion_df_optimism.max_p == "optimism_p"]
emotion_df_opti_anger = emotion_df_optimism[emotion_df_optimism.max_p == "anger_p"]
emotion_df_opti_sad = emotion_df_optimism[emotion_df_optimism.max_p == "sadness_p"]

emotion_df = emotion_df_opti_joy
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_joy, avg_diff_anger, avg_diff_sadness, avg_diff_optimism)

emotion_df = emotion_df_opti_opti
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_joy, avg_diff_anger, avg_diff_sadness, avg_diff_optimism)

emotion_df = emotion_df_opti_anger
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_joy, avg_diff_anger, avg_diff_sadness, avg_diff_optimism)

emotion_df = emotion_df_opti_sad
avg_diff_joy = emotion_df['joy_diff'].sum()/(len(emotion_df))
avg_diff_anger = emotion_df['anger_diff'].sum()/(len(emotion_df))
avg_diff_sadness = emotion_df['sadness_diff'].sum()/(len(emotion_df))
avg_diff_optimism = emotion_df['optimism_diff'].sum()/(len(emotion_df))
print(avg_diff_joy, avg_diff_anger, avg_diff_sadness, avg_diff_optimism)

"""0.30544348894348894 -0.050899999999999994 -0.0357063882063882 -0.21883562653562655
-0.25648516949152544 -0.03857627118644068 0.011552966101694904 0.2835088983050847
-0.1847163265306122 0.5779571428571428 -0.1907107142857143 -0.20252627551020408
-0.23512639186295503 -0.03226161670235546 0.5544996252676659 -0.2871115096359743"""